# **DNA Barcoding Analysis: Alignment-Based vs. Deep Learning Approaches**

In this workshop you will use transformer models as an alternative for BLAST in DNA barcoding

## **Step 1: Setup**

1. Download the metadata from  [metadata](https://drive.google.com/drive/u/1/folders/1Jc57eKkeiYrnUBc9WlIp-ZS_L1bVlT-0)
2. Split the data into pre_training, training, testing and unseen
`python data_split.py BIOSCAN-5M_Dataset_metadata.tsv`

### ***Optional for Collab***

In [ ]:
# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Navigate to the data folder
# %cd /content/drive/My\ Drive/EIFA_Clase_Practica

In [ ]:
# List contents to confirm files are present
# !ls

## **Step 2: Install BLAST**

In [ ]:
# Update and install BLAST
# !sudo apt-get update
# !sudo apt-get install ncbi-blast+

In [ ]:
# Verify BLAST installation
# !blastn -version

## **Step 3: Alignment-Based DNA Barcoding Analysis with BLAST**

In [ ]:
# Create BLAST database
# !makeblastdb -in supervised_train.fas -title train -dbtype nucl -out train.fas

In [ ]:
# Perform BLAST search
# !blastn -query unseen.fas -db train.fas -out results_unseen.tsv -outfmt 6

In [ ]:
# Read BLAST results and extract top hit identifiers
# import pandas as pd
# blast_results = pd.read_csv('results_unseen.tsv', sep='\t', header=None)
# blast_results.columns = [...]
# top_hits = blast_results.groupby('qseqid').first().reset_index()
# top_hits_identifiers = top_hits[['qseqid', 'sseqid']]
# top_hits_identifiers.head()

## **Step 4: Deep Learning-Based DNA Barcoding Analysis**

I am providing you with the driver code to load the transformer models and compute the representations

In [ ]:
# Set up and test transformer models
import sys
import os


sys.path.append('BarcodeBERT-BIOSCAN-5M')
from baselines.datasets import representations_from_df, labels_from_df
from baselines.io import load_baseline_model

data_folder = ""

for model_name in ["DNABERT-S"]: #"BarcodeBERT", "DNABERT-2", "DNABERT-S", "NT",
    if model_name == "BarcodeBERT":
        embedder = load_baseline_model(model_name, checkpoint_path=None, new_vocab=True, k_mer=4, n_heads=6, n_layers=6)
    else:
        embedder = load_baseline_model(model_name)
        embedder.name = model_name
        embedder.model.eval()

        trainable_params = sum(p.numel() for p in embedder.model.parameters() if p.requires_grad)
        print(f"Number of trainable parameters: {trainable_params}")

        embeddings_train = representations_from_df(f"supervised_train.csv", embedder, dataset="BIOSCAN-5M", target="processid")
        print(embeddings_train.shape)

        embeddings_test = representations_from_df(f"unseen.csv", embedder, dataset="BIOSCAN-5M", target="processid")
        print(embeddings_test.shape)


## **Step 5: K-Nearest Neighbors (KNN) Search Using Embeddings**

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Convert embeddings to numpy arrays for KNN
embeddings_train_np = embeddings_train['data'].to_numpy()
embeddings_test_np = embeddings_test['data'].to_numpy()

# Fit KNN model
knn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(embeddings_train_np)

# Find 1-NN for each test embedding
distances, indices = knn.kneighbors(embeddings_test_np)

# Get top identifier for each test instance
closest_ids = [embeddings_train['ids'][i] for i in indices]
print(closest_ids)

## **Step 6: Evaluate your inferred taxonomy**

In [ ]:
### >>>>  For BLAST
# import pandas as pd

# total, correct = 0, 0

# df = pd.read_csv("results_unseen.tsv", sep='\t', header=None, names=["qseqid","sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend","sstart", "send", "evalue", "bitscore"])
# df = df.drop_duplicates(subset=["qseqid"])
# df['qseqid'] = df['qseqid'].apply(lambda x: x[:-1])
# df['sseqid'] = df['sseqid'].apply(lambda x: x[:-1])
# df

# # Load the data from the CSV files
# supervised_df = pd.read_csv('supervised_train.csv')
# unseen_df = pd.read_csv('unseen.csv')

# # Example list of pairs (processid_unseen, processid_supervised)
# pairs = list(zip(df['qseqid'],df['sseqid']))
# #print(pairs)

# # Initialize counters for matches and total pairs
# same_genus_count = 0
# total_pairs = len(pairs)

# # Loop through the pairs and check for genus_name matches
# for unseen_id, supervised_id in pairs:
#     # Find the corresponding genus_name in both DataFrames
#     genus_unseen = unseen_df.loc[unseen_df['processid'] == unseen_id, 'genus_name']
#     genus_supervised = supervised_df.loc[supervised_df['processid'] == supervised_id, 'genus_name']
    
#     # Check if both genus names are found and if they are the same
#     if not genus_unseen.empty and not genus_supervised.empty:
#         if genus_unseen.iloc[0] == genus_supervised.iloc[0]:
#             same_genus_count += 1

# # Calculate the percentage
# percentage_same = (same_genus_count / total_pairs) * 100 if total_pairs > 0 else 0

# print(f"Percentage of matching genus_name values: {percentage_same:.2f}%")

In [ ]:
# >>> ------------- For the transformer model

# import pandas as pd

# total, correct = 0, 0

# pairs = list(zip(embeddings_test['data],closests_ids))
# # Load the data from the CSV files
# supervised_df = pd.read_csv('supervised_train.csv')
# unseen_df = pd.read_csv('unseen.csv')

# # Example list of pairs (processid_unseen, processid_supervised)
# pairs = list(zip(df['qseqid'],df['sseqid']))
# #print(pairs)

# # Initialize counters for matches and total pairs
# same_genus_count = 0
# total_pairs = len(pairs)

# # Loop through the pairs and check for genus_name matches
# for unseen_id, supervised_id in pairs:
#     # Find the corresponding genus_name in both DataFrames
#     genus_unseen = unseen_df.loc[unseen_df['processid'] == unseen_id, 'genus_name']
#     genus_supervised = supervised_df.loc[supervised_df['processid'] == supervised_id, 'genus_name']
    
#     # Check if both genus names are found and if they are the same
#     if not genus_unseen.empty and not genus_supervised.empty:
#         if genus_unseen.iloc[0] == genus_supervised.iloc[0]:
#             same_genus_count += 1

# # Calculate the percentage
# percentage_same = (same_genus_count / total_pairs) * 100 if total_pairs > 0 else 0

# print(f"Percentage of matching genus_name values: {percentage_same:.2f}%")